DM2 Project

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 25.0 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, RobertaConfig
import tensorflow as tf

In [5]:
# Load datasets
hate_speech_df = pd.read_csv(r'/gdrive/My Drive/bullying.csv')
random_comments = pd.read_csv(r'/gdrive/My Drive/Reddit_data.csv')

In [ ]:
hate_speech_df

,Comments
0,stop whining fucking cry baby
1,get off your fucking high horse princess daily...
2,why are you whining stfu you whining sack of shit
3,fucking cry about it you fucking baby
4,you fucking shill stfu
...,...
5053,couple pounds heavier he was fucking 8 pounds ...
5054,fucking cucked mods from here on out prepare y...
5055,fucking mods
5056,you mods are fucking disgraceful


In [ ]:
random_comments = random_comments.dropna(subset=['Comment Body'])

In [ ]:
random_comments

,Unnamed: 0,Comment ID,Author,Timestamp,Comment is a reply to,Comment Thread ID,Thread Title,Comment Body,Comment Permalink,MsgID,ReplyTo,ThreadID,ThreadTitle,MsgBody,Permalink
0,0,jcsq2tx,Joyful_Heretic,3 19 2023 7 22,t3_11vecim,11vecim,I wish I had better friends,I may be wrong here I do that a lot \nPerh...,r Vent comments 11vecim i_wish_i_had_better_f...,NaN,NaN,NaN,NaN,NaN,NaN
1,1,jcsqe0e,Perfect_Scream,3 19 2023 7 26,t1_jcsq2tx,11vecim,I wish I had better friends,I feel like all these friends I was super ...,r Vent comments 11vecim i_wish_i_had_better_f...,NaN,NaN,NaN,NaN,NaN,NaN
2,2,jcsr11s,Joyful_Heretic,3 19 2023 7 36,t1_jcsqe0e,11vecim,I wish I had better friends,I don t know if this will mean anything coming...,r Vent comments 11vecim i_wish_i_had_better_f...,NaN,NaN,NaN,NaN,NaN,NaN
3,3,jcsrd8n,Perfect_Scream,3 19 2023 7 41,t1_jcsr11s,11vecim,I wish I had better friends,ME HUGGING YOU AT THE END https www tiktok...,r Vent comments 11vecim i_wish_i_had_better_f...,NaN,NaN,NaN,NaN,NaN,NaN
4,4,jctvas1,Frecklez1999,3 19 2023 15 04,t3_11vm955,11vm955,My girlfriend being sick is putting a strain o...,Um this is kinda off topic and I m sorry for ...,r Vent comments 11vm955 my_girlfriend_being_s...,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9701,12231,j7sesbk,2 8 2023 18 50,yargspongebob,10xc13q,Motherhood has stripped me,Thank you so much,j7sdhq6,r Vent comments 10xc13q motherhood_has_stripp...,NaN,NaN,NaN,NaN,NaN,NaN
9702,12232,j7seuu6,2 8 2023 18 51,BeanConsumerThe2nd,10xap3a,Getting more and more depressed as the days go on,A friend of mine advice is to Oh dang I think ...,j7rrwud,r Vent comments 10xap3a getting_more_and_more...,NaN,NaN,NaN,NaN,NaN,NaN
9703,12233,j7sevcy,2 8 2023 18 51,ventingaccount123456,10wsy8i,My mom s kitten is dying,life is unfair Im so sorry that is happening,,r Vent comments 10wsy8i my_moms_kitten_is_dyi...,NaN,NaN,NaN,NaN,NaN,NaN
9704,12234,j7sf3uq,2 8 2023 18 53,yargspongebob,10xc13q,Motherhood has stripped me,You seriously just made me sob with an ugly fa...,j7sb8wp,r Vent comments 10xc13q motherhood_has_stripp...,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
random_comments_df = random_comments[['Comment Body']]

In [ ]:
random_comments_df = random_comments_df.rename(columns={'Comment Body': 'Comments'})

In [ ]:
random_comments_df

,Comments
0,I may be wrong here I do that a lot \nPerh...
1,I feel like all these friends I was super ...
2,I don t know if this will mean anything coming...
3,ME HUGGING YOU AT THE END https www tiktok...
4,Um this is kinda off topic and I m sorry for ...
...,...
9701,j7sdhq6
9702,j7rrwud
9703,
9704,j7sb8wp


In [ ]:
# Assign labels to the datasets: 1 for hate speech, 0 for non-hate speech
hate_speech_df['label'] = 1
random_comments_df['label'] = 0

In [ ]:
random_comments_df

,Comments,label
0,I may be wrong here I do that a lot \nPerh...,0
1,I feel like all these friends I was super ...,0
2,I don t know if this will mean anything coming...,0
3,ME HUGGING YOU AT THE END https www tiktok...,0
4,Um this is kinda off topic and I m sorry for ...,0
...,...,...
9701,j7sdhq6,0
9702,j7rrwud,0
9703,,0
9704,j7sb8wp,0


In [ ]:
# Combine both datasets
combined_df = pd.concat([hate_speech_df, random_comments_df])

In [ ]:
combined_df

,Comments,label
0,stop whining fucking cry baby,1
1,get off your fucking high horse princess daily...,1
2,why are you whining stfu you whining sack of shit,1
3,fucking cry about it you fucking baby,1
4,you fucking shill stfu,1
...,...,...
9701,j7sdhq6,0
9702,j7rrwud,0
9703,,0
9704,j7sb8wp,0


In [ ]:
# Preprocess the text data (remove punctuations, lowercase, etc.)
def preprocess_text(text):
    # Check if the text is a string, otherwise return an empty string
    if isinstance(text, str):
        # Add your custom preprocessing steps here
        return text.lower()
    else:
        return ""


combined_df['Comments'] = combined_df['Comments'].apply(preprocess_text)

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(combined_df['Comments'], combined_df['label'], test_size=0.2, random_state=42)


In [ ]:
# Load RobertaTokenizer  and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
config = RobertaConfig.from_pretrained('roberta-base', num_labels=2)
model = TFRobertaForSequenceClassification.from_pretrained('roberta-base', config=config)


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenize the text data
train_encodings = tokenizer(X_train.tolist(), padding=True, truncation=True, return_tensors='tf')
test_encodings = tokenizer(X_test.tolist(), padding=True, truncation=True, return_tensors='tf')



In [ ]:
# Convert BatchEncoding to dictionary
train_encodings = {key: value.numpy() for key, value in train_encodings.items()}
test_encodings = {key: value.numpy() for key, value in test_encodings.items()}


In [ ]:
# Create TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((train_encodings, y_train.values)).batch(64)
test_dataset = tf.data.Dataset.from_tensor_slices((test_encodings, y_test.values)).batch(64)

In [ ]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

In [ ]:
# Train the model
model.fit(train_dataset, epochs=3, validation_data=test_dataset)

Epoch 1/3


In [ ]:
# Evaluate the model
_, accuracy = model.evaluate(test_dataset)
print(f'Accuracy: {accuracy}')

In [ ]:
# Flag hate speech comments in the Reddit comments dataset
reddit_comments_encodings = tokenizer(random_comments_df['text'].tolist(), padding=True, truncation=True, return_tensors='tf')
threshold = 0.5
y_pred_proba = tf.nn.softmax(model.predict(reddit_comments_encodings)[0], axis=-1)
y_pred = (y_pred_proba[:, 1] > threshold).numpy().astype(int)

In [ ]:
flagged_comments = random_comments_df.reset_index(drop=True)[y_pred == 1]
print("Flagged comments:")
print(flagged_comments)